In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import *

In [2]:
Numsize=500#样本数
feature=165 #特征数
lrate=0.1 #学习速率
batch_size = 100#训练batch的大小
steps=100000#迭代次数
dimension=165#维度
total=400#训练样本数

In [3]:
inputX= pd.read_csv('xDataR1.txt',header=None,sep=',')
inputY=pd.read_csv('yDataR1.txt',header=None,sep=',')

x_data =inputX.values#[:,0:dimension]
y_data=inputY.values#[:,0:dimension]


train_start = 0
train_end = int(np.floor(0.8*Numsize))
test_start = train_end 
test_end = Numsize

xdata_train = x_data[np.arange(train_start, train_end),:]
xdata_test = x_data[np.arange(test_start, test_end), :]
ydata_train =y_data[np.arange(train_start, train_end), :]
ydata_test = y_data[np.arange(test_start, test_end), :]
# print(xdata_test,xdata_test.shape)
# print(ydata_test,ydata_test.shape)
#归一化
xtrainMean=np.mean(xdata_train,axis=0)
xtrainStd=np.std(xdata_train,axis=0)
ytrainMean=np.mean(ydata_train,axis=0)
ytrainStd=np.std(ydata_train,axis=0)

# xtrainMax=np.max(xdata_train,axis=0)
# xtrainMin=np.min(xdata_train,axis=0)
# ytrainMax=np.max(ydata_train,axis=0)
# ytrainMin=np.min(ydata_train,axis=0)


# xdata_train-=xtrainMin
# xdata_train/=(xtrainMax-xtrainMin)

# xdata_test-=xtrainMin
# xdata_test/=(xtrainMax-xtrainMin)

# ydata_train-=ytrainMin
# ydata_train/=(ytrainMax-ytrainMin)

# ydata_test-=ytrainMin
# ydata_test/=(ytrainMax-ytrainMin)

xdata_train-=xtrainMean
xdata_train/=xtrainStd

xdata_test-=xtrainMean
xdata_test/=xtrainStd

ydata_train-=ytrainMean
ydata_train/=ytrainStd

ydata_test-=ytrainMean
ydata_test/=ytrainStd
# print(xdata_train,ydata_train)
#转置
# xdata_train = np.transpose(x_data[np.arange(train_start, train_end), :])
# xdata_test = np.transpose(x_data[np.arange(test_start, test_end), :])
# ydata_train =np.transpose(y_data[np.arange(train_start, train_end), :])
# ydata_test = np.transpose(y_data[np.arange(test_start, test_end), :])


In [4]:
 # 2.定义节点准备接收数据 
# global_step=tf.Variable(0)
# learning_rate=tf.train.exponential_decay(0.1,global_step,100,0.96,staircase=True
xs = tf.placeholder(tf.float32, [None,dimension],name="x_traindata") 
ys = tf.placeholder(tf.float32, [None,dimension],name="y_traindata") 

In [8]:

hidden_layers1 =2048
hidden_layers2=1024
hidden_layers3=512
hidden_layers4=256
# 构建输入层到隐藏层,假设隐藏层有 hidden_layers 个神经元
w1=tf.Variable(tf.random_normal([dimension,hidden_layers1]))
b1= tf.Variable(tf.constant(0.0, shape=[1,hidden_layers1], dtype=tf.float32))
h1=tf.nn.relu( tf.add(tf.matmul(xs,w1), b1))

w2=tf.Variable(tf.random_normal([hidden_layers1,hidden_layers2]))
b2= tf.Variable(tf.constant(0.0, shape=[1,hidden_layers2], dtype=tf.float32))
h2=tf.nn.relu(tf.add( tf.matmul(h1,w2), b2))

w3=tf.Variable(tf.truncated_normal([hidden_layers2,dimension]))
b3= tf.Variable(tf.constant(0.0, shape=[1,dimension], dtype=tf.float32))
prediction=tf.add( tf.matmul(h2,w3), b3)

# w1,h1 = add_layer("1",xs, dimension, hidden_layers1,"w1","b1",activation_function=None)#None
# # h1=tf.nn.relu(h1)
# w2,h2 = add_layer("2",h1, hidden_layers1, hidden_layers2,"w2","b2", activation_function=None)
# # w3,h3 = add_layer("3",h2, hidden_layers2, hidden_layers3, "w3","b3",activation_function=None)
# # w4,h4= add_layer("4",h3, hidden_layers, hidden_layers,"w4","b4", activation_function=tf.nn.tanh)
# # # 构建隐藏层到输出层
# w5,prediction = add_layer("3",h2, hidden_layers2 ,dimension,"w3","b3", activation_function=None)


In [13]:
loss =tf.reduce_mean((tf.abs(prediction-ys)+tf.abs(prediction-xs)))
#tf.reduce_sum(tf.abs(-prediction))#tf.losses.mean_squared_error(ys, prediction)#)#-tf.reduce_mean(ys*tf.log(tf.clip_by_value(prediction,1e-10,1)))#(tf.abs(tf.subtract(prediction,ys)))#reduce_sum(tf.square(ys - prediction), reduction_indices=[1]))
#loss=-tf.reduce_mean()tf.log(
train_step =tf.train.AdamOptimizer(0.001).minimize(loss)
#tf.train.GradientDescentOptimizer(lrate).minimize(loss) # GradientDescentOptimizer 优化算法选取SGD,随机梯度下降

# print('将计算图写入事件文件,在TensorBoard里查看')
# writer = tf.summary.FileWriter(logdir='logs/8_2_BP', graph=tf.get_default_graph())
# writer.close()

In [14]:
cost_history=[ ]
saver = tf.train.Saver()
test={xs: xdata_test, ys: ydata_test}
testx={xs: xdata_test}
trainx={xs:xdata_train}
train={xs: xdata_train, ys: ydata_train}
with tf.Session() as sess:

    # 初始化所有变量
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(1,steps+1):
    #     batchSize使用
    #     start=(i*batch_size)%total
    #     end=min(start+batch_size,total)
    #     if start==end:
    #         start=398
    #         end=399  
        sess.run(train_step, feed_dict=train)
        cost=sess.run(loss, feed_dict=train)
        if i % 100 == 0:
    #             m=(i)%100
    #             n=(i+1)%100
    #             if m>n:
    #                 m=n-1
#             print(train)
           
#             hid1=sess.run(h1,feed_dict=trainx)
#             print("\nhidden1:",hid1) 
#             xw=sess.run(tf.matmul(hid1,w2),feed_dict=trainx)
#             print("No jihuo:\n",xw)
#             hid2=sess.run(h2,feed_dict=trainx)
#             print("\nhidden2:",hid2)
# #             wei1=sess.run(w1,feed_dict=testx)
#             print("\nw1:",wei1)
            #p1=sess.run()
            predict=sess.run(prediction,feed_dict=train)
            print('prediction \n',predict*ytrainStd+ytrainMean)
            cost_history.append(cost) 
    predict=sess.run(prediction,feed_dict={xs:xdata_test})
    #如果归一化了，结果得反归一predict=predict*ytrainStd+ytrainMean
    np.savetxt('predict.csv', predict*ytrainStd+ytrainMean, delimiter = ',')
    error=(predict-ydata_test)*ytrainStd
    np.savetxt('error.csv', error, delimiter = ',')
    saver.save(sess,"./model.ckpt")
    writer=tf.summary.FileWriter("./log",tf.get_default_graph())
    writer.close()
    print(np.min(cost_history))

    plt.plot ( range ( len ( cost_history ) ) ,cost_history )

    plt.axis ( [ 0,steps/100,0,10] )

    plt.xlabel ( 'training epochs' )

    plt.ylabel ( 'cost' )

    plt.title ( 'cost history' )

    plt.show ( ) 

# with tf.Session() as sess:
#     saver.restore(sess,"../model.ckpt")
    
#     print(sess.run(prediction,feed_dict={xs:xdata_train})

prediction 
 [[  2552.93575249   -331.54632768  28701.72144701 ...  -1844.41129844
   -2675.80319966 -19715.51491636]
 [   345.94304784  10557.42154685 -17672.48736363 ...  -3409.34060086
    6281.61817967  -6378.68695158]
 [  8165.70804823  -2632.7325153   12505.64655805 ...   9904.52259443
    -498.5277547   21367.04940087]
 ...
 [ -3183.90006223   -866.84630646     47.93828223 ... -12954.40435729
  -12808.19159409   9021.29223813]
 [   -56.32099443  24395.24071365 -28606.07043271 ...   -177.3465354
  -14406.56605026  11833.10710633]
 [ -9935.1557909   -7612.60198747  39850.69494495 ...   1581.74546613
    6593.90985232  19150.92082957]]
prediction 
 [[ 2.32633695e+03  2.63603900e+03  2.20840074e+04 ...  1.25219244e+01
   2.10303786e+03 -1.02839355e+04]
 [ 1.68199093e+02  6.50273558e+03 -5.25655911e+03 ... -1.38724705e+03
  -1.24422409e+02 -9.87744557e+02]
 [ 4.35272673e+03 -4.42627467e+01  6.55586206e+03 ...  5.77668003e+03
  -9.00410963e+02  1.43941534e+04]
 ...
 [-2.32772949e+03 -

prediction 
 [[ 1.32078575e+02  4.56697234e+00  1.88694483e+03 ...  8.98507930e+01
  -1.47886573e+01 -2.01480166e+02]
 [-9.87433618e-01 -4.06910263e+02 -5.58533646e+02 ... -8.25012118e+02
  -2.68870500e+02  6.52150717e+02]
 [ 3.42244922e+01 -3.64372709e+02 -2.38491565e+02 ...  6.13886098e+02
  -1.04252300e+02 -2.81063763e+02]
 ...
 [ 1.42018143e+02 -3.81284157e+02  5.87192032e+01 ...  1.96984596e+02
  -5.03092624e+02  4.60348483e+02]
 [ 9.33929872e+01  5.05318635e+03 -7.23930769e+03 ...  2.77385777e+02
   2.06498548e+02 -6.82125564e+02]
 [ 6.51759065e+01  1.14555938e+02 -2.08262962e+02 ... -1.95735863e+01
   1.71505903e+03 -2.51727207e+02]]
prediction 
 [[ 1.91665094e+02 -4.35604603e+01  1.59737505e+03 ... -5.17854732e+01
  -3.19477550e+02 -2.76280534e+02]
 [ 1.86735532e+02 -2.39553954e+02 -4.83467256e+02 ... -5.73157651e+02
  -2.39466775e+02 -1.97469658e+02]
 [ 1.95201784e+02 -1.91052789e+02 -2.12976996e+02 ...  6.83351703e+02
  -2.55536587e+02 -4.26051991e+02]
 ...
 [ 2.73319458e+02 

prediction 
 [[ 109.52736802   30.26554727  -53.18693484 ...    4.50936022
  -109.94958348 -200.98816788]
 [ -64.97453027   -4.22258854  263.51770045 ...  -17.87861137
   -37.57657791  351.63595548]
 [  -2.12386767 -128.75251657   88.90411678 ...   91.01625612
  -126.56074796  149.61548755]
 ...
 [  37.69580907  -32.28428985  252.67144429 ...   14.10588412
    16.09714239  127.21705946]
 [ 133.59229052 1247.30342072 -641.475591   ...  -50.46903969
   166.74589374 -657.72878838]
 [   1.78325864  140.02044283 -634.93504595 ...  -90.57073541
   104.70392774 -336.57948686]]
prediction 
 [[ 185.93124603 -172.50135952  260.92160314 ...   61.46850457
  -193.16075558  -20.51393777]
 [  77.34099387   -6.2295157   225.57267501 ...   78.66197175
   202.30173002   33.30533026]
 [ 132.72443609 -141.72227115  189.39139633 ...  121.721662
   -74.05141221  -54.61817538]
 ...
 [ 210.48313843 -317.92819682   19.57577683 ...  -33.00513145
    41.7536686  -134.69284646]
 [ 323.48033367  664.26226985 -247.

prediction 
 [[ 192.01076088 -219.23815546  121.91102979 ...   24.9885803
  -166.95273685 -213.97046007]
 [ 318.26442529 -365.71000983   20.42696611 ...   83.83975539
  -191.59656225 -141.08633463]
 [ 203.11717219 -243.48412005 -152.45065481 ...   93.86967455
  -155.31166934 -112.54344321]
 ...
 [ 251.60834347 -328.1727603    10.45736169 ...   60.32268741
  -349.83053429 -255.81494664]
 [ 207.76114316 -146.64808792   -3.00924905 ...   75.04033974
  -155.08254984 -397.75151617]
 [ 199.71295663 -201.98616836 -360.36703064 ...   96.59907334
  -144.63515043   99.31351455]]
prediction 
 [[ 115.97373087  -38.51629137  -69.98258932 ...  128.04646097
   -33.58637481 -151.29273041]
 [  90.21234947   78.22950198  598.78371028 ...  101.70506962
    99.1771499   355.02696477]
 [  94.78044999  -81.66539722  200.5866722  ...   92.91186797
     5.07371478  159.04842796]
 ...
 [  79.85025089 -162.87177864  280.46684513 ...   86.75802827
  -147.97945231  358.45368438]
 [  24.24582148  -75.07529917 -247

prediction 
 [[  60.21117602 -228.08111597  224.88213643 ...  -15.41841828
  -235.28426998  373.52377787]
 [ -61.12518118 -284.37280857  281.8046713  ...  159.55093556
  -190.82969985  -32.07254275]
 [  20.39883494 -184.35281174  173.18626542 ...   21.87177663
   -70.17941203   84.58730786]
 ...
 [  71.39372682 -314.40111235  191.27099063 ...  113.17535102
  -202.53186238   59.95284481]
 [ -93.173047   -352.45421007  389.81081273 ...   22.91824008
  -402.67448962  525.3003164 ]
 [ -48.11460995  -61.70126816  157.49442253 ...   24.10788987
  -289.01235     419.61253187]]
prediction 
 [[ 134.96017821  -52.55787283  140.68877463 ...  118.91114246
  -119.03048804  101.86351744]
 [  88.58443272   30.44353689  213.51216031 ...  125.05585948
   -82.09822375  217.27481634]
 [ 105.41815592  -43.64414817   69.86831431 ...   76.82599195
  -161.07324203  133.39146892]
 ...
 [ 129.86340595  -21.74556092  -75.37078081 ...   76.35148267
   111.49338989  -23.72946708]
 [ 185.65159146  121.68817174  27

...

prediction 
 [[  62.57750335 -114.85128102  127.63684548 ...   51.0327978
  -233.28650098  221.19062934]
 [  43.26416527   -1.67836534  120.65612825 ...   96.25399822
  -340.75837088   44.74213122]
 [ 108.65517946 -107.64546481   86.2444066  ...   67.45615911
  -191.61106823  107.49287971]
 ...
 [  73.74848525  -94.29170614  132.90340225 ...   73.14454403
  -298.7073057   264.80984921]
 [  45.41302085 -105.37639273  261.73720951 ...  113.41296202
  -246.93079171   99.90768926]
 [  -1.5292633    -9.94931456  200.51045215 ...  142.68749605
  -310.50111954  216.2603566 ]]
prediction 
 [[ 184.4930033  -106.83346794  109.24356469 ...  101.49517784
  -108.29996873  135.06589185]
 [ 133.18016324  -30.68723652  293.7694984  ...  125.84901994
  -146.23047698  290.79755917]
 [ 155.48035899 -173.98678437   91.00743797 ...   99.82288562
  -152.77829667  112.46703249]
 ...
 [ 222.24689184 -120.63928292  168.56710011 ...  109.8133393
  -163.1826243   192.87185167]
 [ 265.61236862  -20.85356427  304.

prediction 
 [[  94.30618822 -195.82950239  125.5411463  ...   69.67561355
   -54.86902805  175.64001013]
 [ 178.21924278 -143.79196978  220.12129762 ...  -22.58445355
  -235.41923568  294.11351942]
 [ 165.69745749  -97.58737042  178.1889889  ...   -4.91925149
  -179.78604177   88.94907483]
 ...
 [ 101.62607225 -266.55085872  170.68390549 ...  -39.27802042
  -157.4366158   245.24767483]
 [   1.55725316 -329.96586893  223.50456737 ...   28.43708701
   -10.59415598  462.76502608]
 [  75.16100904 -181.54699701  291.42402136 ...   92.04778367
   -50.10019772  140.32451569]]
prediction 
 [[ 125.70191889  -88.59710033   30.81485344 ...   95.76027471
  -115.68866364  -67.34325577]
 [ 199.0533781  -176.98290233  340.49961929 ...  125.4174364
    -4.13666441  -60.83006344]
 [ 148.35684776 -125.32130981  156.57485456 ...   80.73681443
   -91.62707132 -110.46587274]
 ...
 [ 118.92523499 -229.65001213  208.39922068 ...   91.00405537
   -70.60221612  -44.79951773]
 [ 225.26829961    6.60912346  121

Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x7fcffaeaae70> >>
Traceback (most recent call last):
  File "/home/iloyu/tensorflow/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 970, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt


prediction 
 [[ 123.5407351  -106.49076642   75.892646   ...   64.93254318
  -209.76037195  -10.76849199]
 [ 151.97305118  -29.63488972  371.04877349 ...   88.01274737
   -29.37009872 -235.94614589]
 [ 119.40381145  -93.47885342  145.28771076 ...   88.21926359
  -152.55153751   23.27511847]
 ...
 [ 111.62530805   -1.96331922  129.12578466 ...   13.11610892
   -29.10627435  -27.85285341]
 [ 134.18143099   -0.59780638  185.69344918 ...   19.57245126
   -61.69032404 -101.63609202]
 [  54.29361314  -51.67293609  336.61933256 ...   54.77619666
  -278.02945352 -129.38201209]]
prediction 
 [[ 136.09931007  -50.85754674   64.68272331 ...   13.46867059
  -203.1623341    10.93210104]
 [ 193.70293925   35.7209168   321.88064986 ...   80.38723168
   -62.00270995   15.27704627]
 [ 143.64620009 -100.23708677   49.26007969 ...   68.84612146
  -125.88667668   94.56063361]
 ...
 [ 209.69041667    0.65841869  109.41928045 ...   74.53457249
   -47.56391367  -17.90106293]
 [ 192.32281005   -3.37465621  26

KeyboardInterrupt: 

In [ ]:
predict.shape

In [ ]:
def add_layer(n_layer,inputs, in_size, out_size,Wname,Bname, activation_function):
    # 构建权重 : in_size * out)_sieze 大小的矩阵
    layer_name = 'layer%s'% n_layer
    with tf.name_scope('layer_name'):
        with tf.name_scope('Weights'):
            weights = tf.Variable(tf.truncated_normal([ in_size,out_size]),name=Wname)
            # 构建偏置 : 1 * out_size 的矩阵
            biases = tf.Variable(tf.constant(0.0, shape=[1,out_size], dtype=tf.float32),name=Bname)
            # 矩阵相乘
            Wx_plus_b =tf.add( tf.matmul(inputs,weights), biases)
            if activation_function is None:
                outputs = Wx_plus_b
            else:
                outputs = activation_function(Wx_plus_b)

    return weights,outputs  # 得到输出数据

In [ ]:
w = tf.Variable(tf.random_normal([165,165]))
b = tf.Variable(tf.zeros([1,165])+0.1)
prediction = tf.nn.softmax(tf.add(tf.matmul(xs,w),b))
loss=tf.reduce_mean(tf.square(ys-prediction))
train=tf.train.GradientDescentOptimizer(lrate).minimize(loss)

In [ ]:
# x_data=np.linspace(-1,1,165)[400]#[:,np.newaxis] #300*1,输入只有一个神经元
#     X = tf.placeholder (tf.float32, shape = [None, input_size])
#     Y = tf.placeholder (tf.float32, shape = [None, num_classes])import tensorflow as tf



# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)
# def bias_variable(shape):
#     initial = tf.zeros(shape)
#     return tf.Variable(initial)

# def add_layer(inputs,in_size,out_size,activation_function=None):
#     Weights=weight_variable([in_size,out_size])
#     biases=bias_variable([1,out_size])
#     Wx_plus_b=tf.matmul(inputs,Weights)+biases
#     if activation_function==None:
#         outputs=Wx_plus_b
#     else:
#         outputs=activation_function(Wx_plus_b)
#     return [outputs,Weights]

 # 3.定义神经层：隐藏层和预测层 
 
h1=18
h2=36
h3=72
l_1 = add_layer(xs,dimension,h1,activation_function = tf.nn.relu)
# l_2=add_layer(l_1,h1,h2,activation_function = tf.nn.relu)
prediction = add_layer(l_1,h1,dimension,activation_function = tf.nn.sigmoid)

# l1 = add_layer(xs,dimension,h1,activation_function=tf.nn.relu) 
# # w2,l2=add_layer(l1,h1,h2,activation_function=tf.nn.relu)
# # w3,l3=add_layer(l2,h2,h3,activation_function=tf.nn.relu)
# prediction=add_layer (l1,h1,dimension,activation_function=tf.nn.relu) 

loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1])) 

train_step =tf.train.GradientDescentOptimizer(lrate).minimize(loss)#tf.train.AdamOptimizer(lrate).minimize(loss) # 
# valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
# important step 对所有变量进行初始化 
init = tf.global_variables_initializer()#tf.initialize_all_variables() 
sess = tf.Session() 
# 上面定义的都没有运算，直到 sess.run 才会开始运算 
sess.run(init) 
# 迭代 1000 次学习，
cost_history= [ ] 
for i in range(steps+1): 

    start = max((i * batch_size) % (Numsize),0)
    end = min(start + batch_size,Numsize)
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data}) 
    if i % 50 == 0:
# to see the step improvement
        print ( "After %d iteration:" %i ) 
        cost=sess.run(loss, feed_dict={xs: x_data, ys: y_data})
        print(cost)
#         print(sess.run(w,feed_dict={xs: x_data, ys: y_data}))
        cost_history.append (cost) 
saver = tf.train.Saver()
# print(cost_history)
plt.plot ( range ( len ( cost_history ) ) ,cost_history )

plt.axis ( [ 0,10,0,np.max ( cost_history ) ] )

plt.xlabel ( 'training epochs' )

plt.ylabel ( 'cost' )

plt.title ( 'cost history' )

plt.show ( ) 
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
#     save_path = saver.save(sess, "my_net/save_net.ckpt")
#     print("Save to path: ", save_path)
# pred_y = sess.run ( y, feed_dict={x: X_test} )

# mse = tf.reduce_mean ( tf.square ( pred_y - y_test ) )

# print ( "MSE: %.4f" % sess.run ( mse ) ) 


In [ ]:
# import tensorflow as tf 
# from  tensorflow.examples.tutorials.mnist  import  input_data
import numpy as np 
from numpy import *
x_data=np.linspace(1,165)#reshape(400,165)#np.linspace(-1,1,165)
x_data=mat(random.randint(50,200,size=(400,165)))
# mnist = input_data.read_data_sets('data', one_hot = True)

In [ ]:
x_data.shape

In [ ]:
np.ones(Numsize*feature)

In [ ]:
# x_data=mat(np.ones(Numsize*feature),size=(Numsize,feature))
x_data=np.ones((total,feature))
#x1=x1.reshape(Numsize,feature)
print(x_data[400:500])
# x_data=mat(random.randint(50,200,size=(Numsize,feature)))
# noise=np.random.normal(0,0.05,x_data.shape)
y_data=x_data*0.3+0.1#np.square(x_data)-0.5#+noise
print(y_data)
xs=tf.placeholder(tf.float32,[None,feature])
ys=tf.placeholder(tf.float32,[None,feature])
#定义隐含层
# l1=add_layer(xs,165,165,activation_function=tf.nn.relu)
# l2=add_layer(l1,165,165,activation_function=tf.nn.relu)

#定义输出层,假设没有任何激活函数
# [prediction,w]=add_layer(xs,165,165,activation_function=None)
Weights = tf.Variable(tf.random_uniform([1], -1.0, 1),(165,1))
biases = tf.Variable(tf.zeros([1]),(1,165))
prediction = Weights*xs + biases
loss=tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction)))#,reduction_indices=[1]
optimizer=tf.train.GradientDescentOptimizer(lrate)
train=optimizer.minimize(loss)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):

#         print(start,end)
        sess.run(train,feed_dict={xs:x_data,ys:y_data})
        if i%100==0:
            print(sess.run(loss,feed_dict={xs:x_data,ys:y_data}))
            print(sess.run(Weights,feed_dict={xs:x_data,ys:y_data}))
            

In [ ]:
plt.figure()
for i in range(399):
    plt.scatter(xdata_train[i,:],ydata_train[i,:],marker='o')
    plt.xlabel('x')
    plt.ylabel('y')
plt.show()